In [1]:
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import re

# Load the same sentence-transformer model used in our Lambda for consistency.
print("Loading the sentence-transformer model 'all-MiniLM-L6-v2'...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded successfully.")

/opt/anaconda3/envs/runic/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading the sentence-transformer model 'all-MiniLM-L6-v2'...
Model loaded successfully.


In [3]:
def parse_and_chunk_markdown(doc):
    """Parses a markdown document, splitting it into chunks with section metadata."""
    chunks = []
    lines = doc['content'].split('\n')

    current_section = "Introduction" # Default section
    current_chunk_lines = []

    for line in lines:
        # Check for a markdown header (e.g., "## 🔍 Symptoms")
        header_match = re.match(r'^##\s+(.*)', line)
        if header_match:
            # If we find a new header, save the previous chunk
            if current_chunk_lines:
                chunks.append({
                    'parent_doc': doc['name'],
                    'section': current_section,
                    'content': "\n".join(current_chunk_lines).strip()
                })
                current_chunk_lines = [] # Reset for the new section

            # Update the current section title (stripping the emoji)
            current_section = header_match.group(1).split(' ', 1)[-1].strip()

        # Add non-empty lines to the current chunk
        if line.strip():
            current_chunk_lines.append(line)

    # Add the last remaining chunk
    if current_chunk_lines:
        chunks.append({
            'parent_doc': doc['name'],
            'section': current_section,
            'content': "\n".join(current_chunk_lines).strip()
        })

    return chunks

def chunk_generic_text(doc):
    """Splits generic text (like logs or JSON) into simple, overlapping chunks."""
    # For non-markdown files, we can just treat them as single-section documents
    return [{
        'parent_doc': doc['name'],
        'section': 'Log Content',
        'content': doc['content']
    }]


def load_and_chunk_documents(path):
    """Loads all documents and processes them into a unified list of chunks."""
    all_chunks = []
    for root, _, files in os.walk(path):
        for file_name in files:
            if file_name.endswith(('.log', '.json', '.txt', '.md')):
                file_path = os.path.join(root, file_name)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        doc = {'name': file_name, 'content': f.read()}

                        if file_name.endswith('.md'):
                            chunks = parse_and_chunk_markdown(doc)
                        else:
                            chunks = chunk_generic_text(doc)

                        all_chunks.extend(chunks)
                except Exception as e:
                    print(f"Error reading or chunking {file_path}: {e}")
    return all_chunks

# --- Execute the loading and chunking ---
print("\nLoading, parsing, and chunking all documents from '../data'...")
all_chunks = load_and_chunk_documents('../data')
print(f"Successfully created {len(all_chunks)} chunks from the document base.")


Loading, parsing, and chunking all documents from '../data'...
Successfully created 70 chunks from the document base.


In [4]:
print("\nCreating embeddings for all chunks... (This may take a moment)")
chunk_contents = [chunk['content'] for chunk in all_chunks]

# We encode the content of each chunk, not the whole document
chunk_embeddings = model.encode(chunk_contents, convert_to_tensor=False, show_progress_bar=True)

print(f"Embeddings created successfully. Vector shape: {chunk_embeddings.shape}")


Creating embeddings for all chunks... (This may take a moment)


Batches: 100%|██████████| 3/3 [00:01<00:00,  2.38it/s]

Embeddings created successfully. Vector shape: (70, 384)


In [5]:
# The dimension 'd' is the size of our embeddings (384 for 'all-MiniLM-L6-v2')
d = chunk_embeddings.shape[1]

print("\nBuilding the FAISS index from chunk embeddings...")
index = faiss.IndexFlatL2(d)
index.add(np.array(chunk_embeddings, dtype='float32')) # Ensure dtype is float32 for FAISS
print(f"FAISS index built. Total vectors in index: {index.ntotal}")


Building the FAISS index from chunk embeddings...
FAISS index built. Total vectors in index: 70


In [6]:
def search(query, k=3):
    """Searches the FAISS index and returns the top k most relevant CHUNKS."""
    print(f"\n================================================================")
    print(f"Searching for top {k} chunks matching query: '{query}'")
    print(f"================================================================")

    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding, dtype='float32'), k)

    print("\n--- Search Results ---")
    for i, idx in enumerate(indices[0]):
        # Retrieve the chunk using the index
        retrieved_chunk = all_chunks[idx]

        print(f"\n{i+1}. Document: '{retrieved_chunk['parent_doc']}' | Section: '{retrieved_chunk['section']}' (Score: {distances[0][i]:.4f})")
        print("--------------------------------------------------")
        # Print the actual content of the chunk
        print(retrieved_chunk['content'])

# --- Execute Test Queries ---
search("The auth-service has high CPU usage and is exhausted")
search("I'm getting database connection timeouts and latency spikes from the payment gateway")
search("My search-engine service is in a crash loop after the last deployment and is unavailable")


Searching for top 3 chunks matching query: 'The auth-service has high CPU usage and is exhausted'

--- Search Results ---

1. Document: 'high_cpu_restart.md' | Section: 'Resolution Steps' (Score: 0.8212)
--------------------------------------------------
## 🛠️ Resolution Steps
1.  **Acknowledge Incident:** Update the incident status in the `OpsFlowIncidents` table to `INVESTIGATING`.
2.  **Immediate Mitigation:** Perform a rolling restart of the affected service (e.g., `auth-service`). This is the fastest way to recover from a stuck process.
3.  **Monitor Post-Restart:** Closely observe the `cpuUsagePercent` metric for 5 minutes after the restart.
4.  **Escalate if Unresolved:** If the CPU spikes again immediately, the issue is likely not a transient stuck process. Escalate to the on-call engineer and consider a rollback of the last deployment.

2. Document: 'incident_001_cpu_spike.json' | Section: 'Log Content' (Score: 0.8946)
--------------------------------------------------
{
  "t